In [2]:
import pandas as pd
from nsepy import get_history as gh
import datetime as dt

In [3]:
import yfinance as yf
import datetime as dt

symbol = 'TATACOFFEE.NS'
start = dt.datetime(2021,6,1)
end = dt.datetime(2022,1,28)

# Retrieve historical stock data from Yahoo Finance
stk_data = yf.download(symbol, start=start, end=end)

[*********************100%%**********************]  1 of 1 completed


In [4]:
stk_data=stk_data[["Open","High","Low","Close"]]
#stk_data.to_csv("Tatacoffee13_21.csv")

In [5]:
from sklearn.preprocessing import MinMaxScaler
Ms = MinMaxScaler()
data1= Ms.fit_transform(stk_data)
print("Len:",data1.shape)

Len: (166, 4)


In [6]:
data1=pd.DataFrame(data1,columns=["Open","High","Low","Close"])

In [7]:
training_size = round(len(data1 ) * 0.80)
print(training_size)
X_train=data1[:training_size]
X_test=data1[training_size:]
print("X_train length:",X_train.shape)
print("X_test length:",X_test.shape)
y_train=data1[:training_size]
y_test=data1[training_size:]
print("y_train length:",y_train.shape)
print("y_test length:",y_test.shape)

133
X_train length: (133, 4)
X_test length: (33, 4)
y_train length: (133, 4)
y_test length: (33, 4)


In [8]:
import warnings
warnings.filterwarnings("ignore")

In [9]:
performance={"Model":[],"RMSE":[],"MaPe":[],"Lag":[],"Test":[]}

In [13]:
listt=["Close","High","Open","Low"]

#listt=["AQI_calculated","PM10","PM2.5","NOx","NO2","NO","NH3","SO2","CO",'year']
dataset=stk_data

In [14]:
print(listt)
datasetTwo=dataset[listt]
test_obs = 28
train =datasetTwo[:-test_obs]
test = datasetTwo[-test_obs:]

['Close', 'High', 'Open', 'Low']


In [15]:
from statsmodels.tsa.api import VAR
for i in [1,2,3,4,5,6,7,8,9,10]:
    model = VAR(train)
    results = model.fit(i)
    print('Order =', i)
    print('AIC: ', results.aic)
    print('BIC: ', results.bic)
    print()

Order = 1
AIC:  7.294295354735824
BIC:  7.720569942447959

Order = 2
AIC:  7.431614246492745
BIC:  8.202611128011112

Order = 3
AIC:  7.591960268586673
BIC:  8.711029072133327

Order = 4
AIC:  7.677532774333513
BIC:  9.148078344457856

Order = 5
AIC:  7.750415380263846
BIC:  9.575899040193374

Order = 6
AIC:  7.883366827323744
BIC:  10.067307677767964

Order = 7
AIC:  8.029445577293282
BIC:  10.575421832952317

Order = 8
AIC:  8.238946288901966
BIC:  11.150596653979942

Order = 9
AIC:  8.391204988933644
BIC:  11.6722300730075

Order = 10
AIC:  8.290443394524745
BIC:  11.944607170171754



C:\Users\kmuth\anaconda3\envs\myenv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\kmuth\anaconda3\envs\myenv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\kmuth\anaconda3\envs\myenv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\kmuth\anaconda3\envs\myenv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored

In [16]:
x = model.select_order(maxlags=12)
order=x.selected_orders["aic"]
result = model.fit(order)
#result.summary()
lagged_Values = train.values[-order:]
pred = result.forecast(y=lagged_Values,steps=28) 
preds=pd.DataFrame(pred,columns=listt)
preds.to_csv("varforecasted_{}.csv".format(test_obs))

In [17]:
from sklearn.metrics import mean_squared_error
rmse= round(mean_squared_error(test,pred,squared=False))

In [18]:
from sklearn.metrics import mean_absolute_percentage_error
mape=mean_absolute_percentage_error(test,pred)
performance["Model"].append(listt)
performance["RMSE"].append(rmse)
performance["MaPe"].append(mape)
performance["Lag"].append(order)
performance["Test"].append(test_obs)
perf=pd.DataFrame(performance)

In [19]:
data1

,Open,High,Low,Close
0,0.140240,0.072381,0.073518,0.021463
1,0.049331,0.000000,0.064032,0.017488
2,0.096547,0.038730,0.120158,0.060413
3,0.110641,0.028571,0.090909,0.023053
4,0.089500,0.012698,0.079051,0.023053
...,...,...,...,...
161,0.696970,0.619048,0.796838,0.744038
162,0.671600,0.619048,0.769960,0.741653
163,0.675828,0.563175,0.588142,0.558029
164,0.527836,0.483175,0.505929,0.620032


In [20]:
perf

,Model,RMSE,MaPe,Lag,Test
0,"[Close, High, Open, Low]",9,0.038877,1,28
